# Sequential and Parallel Composition

## Introduction

In this notebook, we will: 
- Define sequential composition and parallel composition.
- Illustrate sequential composition and parallel composition in practice through polars queries.
- Compare and contrast sequential and parallel composition.

Sources: [Textbook](https://programming-dp.com/ch4.html), [Paper](https://eprint.iacr.org/2021/1196.pdf)

### Composition


Before we get into the different types of composition, let's understand why composition is important and what it means. We will rarely only want one differentially private statistic; instead, we will compute multiple statistics on the same dataset. 

If we run many different differentially private algorithms on the same dataset, the resulting **composed** algorithm is also differentially private. So composition is a useful tool because it enables us to analyze the privacy properties of algorithms designed in this way. 

### Key Terms

1. **Noninteractive:** 

All queries are composed and executed at once. Specifically, take $F$ as a noninteractive algorithm composed of individual queries $F = \text{Comp}(F_1, F_2, \ldots, F_{k-1})$ and dataset x as the input dataset. $F(x)$ is our output, an algorithm that executes all the queries. 

2. **Interactive:** 

In an interactive algorithm, the queries are not defined in advance, and they can be determined based on the results from previous queries. This is an **adaptive** scenario. 

A non-adaptive scenario is where you simply want to compute statistics such as the mean, median, and sum of particular columns in a dataset. An adaptive scenario is where you start computing particular statistics about a column, such as "income," and then change your queries depending on the results you find. For example, if you notice that the mean and median are extremely different, you may explore the distribution of the data. Otherwise, you may choose to focus your queries on other, more interesting aspects of your data. 

Interactive differential privacy is considered the "right" modeling for many applications since it captures the full capabilities of many fundamental DP mechanisms such as the Sparse Vector Technique. 

3. **Signal:**
A large count = strong signal. A dataset with a large count likely won't be disrupted by adding weak noise, so higher utility (usefulness). 

## Set Up

In [1]:
pip install "opendp[polars]"


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import opendp.prelude as dp
import numpy as np
import polars as pl 

dp.enable_features("contrib")

In [3]:
df = pl.scan_csv("sample_FR_LFS.csv", infer_schema_length=1000, ignore_errors=True)

## Sequential Composition

In [4]:
context = dp.Context.compositor(
    data=df,
    privacy_unit=dp.unit_of(contributions=36),
    privacy_loss=dp.loss_of(epsilon=1.0),
    split_evenly_over=10,
    margins={
        ("ILOSTAT", ): dp.Margin(public_info="lengths", max_partition_length=60_000_000, max_num_partitions=3),
        ("YEAR", "QUARTER", "ILOSTAT",): dp.Margin(public_info="lengths", max_partition_length=60_000_000, max_partition_contributions=1, max_num_partitions=1),
        (): dp.Margin(public_info="lengths", max_partition_length=60_000_000, max_num_partitions=1),
    },
)

Sequential composition is a major property of differential privacy because it bounds the total privacy costs of computing multiple differentially private queries on the same data. It's important because it allows the design of algorithms that refer to the data multiple times. 

It's an extremely useful way to get an **upper** bound on the total ε of many queries, but keep in mind that the actual impact on privacy may be lower. Let's say that function 1, $F1$ satisfies ε-dp and function 2, $F2$, also satisfies ε-dp, then a mechanism where we compose functions $F1$ and $F2$, we get 2 * ε-dp.

Sequential composition can be noninteractive and interactive. 

### Noninteractive Sequential Composition

We can execute multiple queries at once using the `select` or `agg` methods. To start off, let's calculate the actual and differentially private mean and median for the variable `HWUSUAL`, which is the number of hours per week usually worked in the main job by respondents that were working for pay at the time of the survey.

Notes: 
- Remember to reference the [EU Labour Force Survey User Guide](https://ec.europa.eu/eurostat/documents/1978984/6037342/EULFS-Database-UserGuide.pdf) for any questions about variables. 
- Do not calculate the actual mean to impute null values. We shouldn't reference our data after creating the compositor. 

In [5]:
candidates = list(range(25,56))

actual_hours = df.groupby("ILOSTAT").agg(
    pl.col("HWUSUAL").fill_null(40.0).mean().alias("Mean Hours Worked"), 
    pl.col("HWUSUAL").fill_null(40.0).median().alias("Median Hours Worked"),
).collect()

dp_hours = context.query().groupby("ILOSTAT").agg(
    pl.col("HWUSUAL").fill_null(40.0).dp.mean((1,98), scale = 1/97).alias("Mean Hours Worked"), 
    pl.col("HWUSUAL").fill_null(40.0).dp.median(candidates, 1/97).alias("Median Hours Worked"),
).release().collect()

hours_cat_df = actual_hours.join(dp_hours, on=["ILOSTAT"]).filter(pl.col("ILOSTAT")==1)

hours_cat_df

/var/folders/dg/nlwnz_jj7rsblr2tzwj962b40000gn/T/ipykernel_41502/693093094.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  actual_hours = df.groupby("ILOSTAT").agg(


ILOSTAT,Mean Hours Worked,Median Hours Worked,Mean Hours Worked_right,Median Hours Worked_right
i64,f64,f64,f64,i64
1,37.667898,37.0,36.397198,25


## Interactive Sequential Composition

In cases where you want to apply sequential composition to an adaptive scenario, you can use the `dp.c.make_sequential_composition` method. There are two ways to implement sequential composition: using transformations and measurements directly and passing in polars queries. 

### Interactive Sequential Composition with Transformations and Measurements

The following example references code from a pre-existing compositors tutorial. Learn more about it [here](https://docs.opendp.org/en/nightly/api/user-guide/combinators/compositors.html).

The interactive sequential composition doesn't use the compositor we defined at the start of this notebook. Instead, we construct a composition through transformations and measurements, combinator elements present in more lower-level frameworks in the OpenDP library. 

In [6]:
#define the compositor by specifying parameters
sc_meas = dp.c.make_sequential_composition(
    input_domain=dp.vector_domain(dp.atom_domain(T=float)),
    input_metric=dp.symmetric_distance(),
    output_measure=dp.max_divergence(T=float),
    #d_in is the upper bound on distances
    d_in=1,
    #d_mids is the privacy consumption for each query
    d_mids=[1., 1., 2.]
)

#this is the privacy consumption of the entire composition
print("Privacy Consumption of Entire Composition: " + str(sc_meas.map(1)))

#clean and compile data
hours_list = list(np.array(df.select(
                                    pl.col("HWUSUAL").
                                    fill_null(40.0).
                                    fill_nan(40.0).
                                    filter(pl.col("ILOSTAT")==1)).
                                    collect()
                                ).reshape(-1))

#this is a queryable that takes in the data similar to the context
sc_qbl = sc_meas(hours_list)
#we can then apply the queryable to a function of our choosing
sc_qbl 

Privacy Consumption of Entire Composition: 4.0


Queryable(Q=AnyMeasurement)

In [7]:
#construct measurement for mean
#compute dp median as an input for dp mean 
input_space = dp.vector_domain(dp.atom_domain(T=float)), dp.symmetric_distance()
count_meas = input_space >> dp.t.then_count() >> dp.m.then_laplace(scale=1.0)
count_release = sc_qbl(count_meas)
print("DP count: ",count_release)

mean_meas = (
    input_space >> 
    dp.t.then_clamp((0.,79.)) >> 
    dp.t.then_resize(size=count_release, constant=40.) >> 
    dp.t.then_mean() >> 
    dp.m.then_laplace(1.))
print("DP Mean:", sc_qbl(mean_meas))

DP count:  79853
DP Mean: 40.20028441533665


In [8]:
#construct measurement for median
discrete_scores = dp.t.make_quantile_score_candidates(dp.vector_domain(dp.atom_domain(T=float)), 
                                    dp.symmetric_distance(), 
                                    np.array(candidates).astype(float), 
                                    0.5)
input_space = dp.vector_domain(dp.atom_domain(T=dp.usize), 31), dp.linf_distance(T=dp.usize)
select_index_measurement = dp.m.make_report_noisy_max_gumbel(*input_space, scale=1.0, optimize='min')

median_meas = discrete_scores >> select_index_measurement >> (lambda index: candidates[index])
print("DP Median:", sc_qbl(median_meas))

DP Median: 37


That was a lot of code! But as you saw, we could change our queries depending on the outputs of other queries, illustrating interactive sequential composition.

### Interactive Sequential Composition with Polars Queries

You can also implement interactive sequential composition with polars queries! One of the first steps is to specify the domain of our dataframe and add a margin specifying the parameters for the data when it's grouped by `ILOSTAT`. This is **in addition** to specifying it in the compositor. 

In [9]:
#get the domain of the datafarame
df_domain = dp.domain_of(df, infer=True)
#specify the parameters when grouped by Working cateory
df_domain = dp.with_margin(df_domain, by=["ILOSTAT"], 
                           public_info="lengths", 
                           max_partition_length=50, 
                           max_num_partitions=3)
print("DF Domain: ",df_domain)


DF Domain:  FrameDomain(COEFF: f64, QUARTER: i64, REFYEAR: i64, REFWEEK: i64, INTWEEK: f64, COUNTRY: str, DEGURBA: f64, HHINST: f64, INTWAVE: i64, INTQUEST: i64, REM: i64, YEAR: i64, HHPRIV: i64, SEX: i64, AGE: f64, NATIONAL: str, YEARESID: f64, COUNTRYB: str, PROXY: f64, NOWKREAS: i64, STAPRO: f64, SIGNISAL: i64, COUNTRYW: str, YSTARTWK: f64, MSTARTWK: f64, FTPT: f64, TEMP: f64, TEMPDUR: f64, HWUSUAL: f64, HWACTUAL: f64, HWOVERP: f64, HWOVERPU: f64, HOURREAS: f64, WISHMORE: f64, HWWISH: f64, LOOKOJ: i64, EXIST2J: f64, STAPRO2J: f64, HWACTUA2: f64, EXISTPR: f64, YEARPR: f64, MONTHPR: f64, STAPROPR: f64, SEEKWORK: i64, SEEKTYPE: f64, SEEKDUR: f64, METHODA: i64, METHODB: i64, METHODC: i64, METHODD: i64, METHODE: i64, METHODF: i64, METHODG: i64, METHODH: i64, METHODI: i64, METHODJ: i64, METHODK: i64, METHODL: i64, METHODM: i64, WANTWORK: f64, AVAILBLE: i64, EDUCSTAT: f64, EDUCLEVL: f64, COURATT: f64, COURLEN: f64, ILOSTAT: i64, ISCO1D: f64, ISCOPR1D: f64, DURUNE: f64, EDUC4WN: f64, HATLEV

Next, we define our proposed plan, which are the same queries that we worked with in noninteractive sequential composition. However, this time we aren't actually releasing or collecting any results. 

In [10]:
#define polars queries
proposed_plan = context.query().groupby("ILOSTAT").agg(
    pl.col("HWUSUAL").fill_null(40.0).dp.mean((1,98), scale = 8).alias("Mean Hours Worked"), 
    pl.col("HWUSUAL").fill_null(40.0).dp.median(candidates, 8).alias("Median Hours Worked"),
)
proposed_plan

<wrap [3 cols, {"ILOSTAT": Int64 … "Median Hours Worked": Int64}] at 0x16D8417C0>

Next, we set up a measurement to execute the sequential creaties and pass in our `df_domain` and `proposed_plan`. We can pass our data to this measurement and the collect our results. 

In [12]:
# IN SERVER
m_polars = dp.m.make_private_lazyframe(
    input_domain=df_domain, 
    input_metric=dp.symmetric_distance(), 
    output_measure=dp.max_divergence(T=float), 
    lazyframe=proposed_plan, 
    global_scale=1.
)

df_release = m_polars(df).collect().filter(pl.col("ILOSTAT")==1)
print(df_release)
#TODO: more explanation on the different steps here

m_comp = dp.c.make_sequential_composition(
    m_polars.input_domain,
    m_polars.input_metric,
    m_polars.output_measure,
    2, 
    [18.0, 18.0]
)

qbl_comp = m_comp(df)
once_frame = qbl_comp(m_polars)
print(qbl_comp(m_polars))

shape: (1, 3)
┌─────────┬───────────────────┬─────────────────────┐
│ ILOSTAT ┆ Mean Hours Worked ┆ Median Hours Worked │
│ ---     ┆ ---               ┆ ---                 │
│ i64     ┆ f64               ┆ i64                 │
╞═════════╪═══════════════════╪═════════════════════╡
│ 1       ┆ 37.671921         ┆ 29                  │
└─────────┴───────────────────┴─────────────────────┘


## Parallel Composition

Parallel composition can be viewed as an alternative to sequential composition since it's another way to calculate a bound on the total privacy cost of multiple queries. 

Parallel composition involves partitioning the input dataset into disjoint partitions and applying a differentially private mechanism on each chunk independent of all other chunks. Assuming each individual is represented once in the dataset, their data will appear in exactly one chunk. If there are $k$ chunks, there will be $k$ runs of the differentially private mechanisms.

To apply parallel composition, we need to set up our margin or define our query to allow each individual to influence at most 1 partition, or appear at most in 1 chunk. 


### Noninteractive Parallel Composition

In the following example, each individual is represented just one each year, quarter, and labor status. Moreover, this example also satisfies the properties of noninteractive sequential composition since all the queries are executed simultaneously. 

In [17]:
context.query().groupby(["YEAR","QUARTER","ILOSTAT"]).agg(
    pl.col("HWUSUAL").fill_null(74).dp.mean((1,98)).alias("Mean Hours Worked"),
    pl.col("HWUSUAL").fill_null(74).dp.median((25,55)).alias("Median Hours Worked")
).release().collect().filter(pl.col("ILOSTAT")==1).sort(["YEAR", "QUARTER"])

YEAR,QUARTER,ILOSTAT,Mean Hours Worked,Median Hours Worked
i64,i64,i64,f64,i64
2004,1,1,37.231225,25
2004,2,1,41.136433,55
2004,3,1,37.531945,55
2004,4,1,37.375663,55
2005,1,1,35.841911,55
…,…,…,…,…
2012,4,1,38.014647,25
2013,1,1,37.630491,25
2013,2,1,35.786751,55


### Interactive Parallel Composition

We'll illustrate interactive parallel composition using the polars-interactivity methods. Much of the code is similar to interactive sequential composition. The primary change is the domain and proposed plan. 

In [22]:
#get the domain of the datafarame
lf_domain = dp.domain_of(df, infer=True)
lf_domain = dp.with_margin(lf_domain, by=["YEAR","QUARTER","ILOSTAT"], 
                           public_info="lengths", 
                           max_partition_length=50, 
                           max_num_partitions=3)

proposed_plan = context.query().groupby(["YEAR","QUARTER","ILOSTAT"]).agg(
    pl.col("HWUSUAL").fill_null(74).dp.mean((1,98), 8).alias("Mean Hours Worked"),
    pl.col("HWUSUAL").fill_null(74).dp.median((25,55), 8).alias("Median Hours Worked")
)
# IN SERVER
m_polars = dp.m.make_private_lazyframe(
    input_domain=lf_domain, 
    input_metric=dp.symmetric_distance(), 
    output_measure=dp.max_divergence(T=float), 
    lazyframe=proposed_plan, 
    global_scale=1.
)

df_release = m_polars(df).collect().filter(pl.col("ILOSTAT")==1)
print(df_release)

m_comp = dp.c.make_sequential_composition(
    m_polars.input_domain,
    m_polars.input_metric,
    m_polars.output_measure,
    2, 
    [18.0, 18.0]
)


qbl_comp = m_comp(df)
print(qbl_comp(m_polars))

shape: (40, 5)
┌──────┬─────────┬─────────┬───────────────────┬─────────────────────┐
│ YEAR ┆ QUARTER ┆ ILOSTAT ┆ Mean Hours Worked ┆ Median Hours Worked │
│ ---  ┆ ---     ┆ ---     ┆ ---               ┆ ---                 │
│ i64  ┆ i64     ┆ i64     ┆ f64               ┆ i64                 │
╞══════╪═════════╪═════════╪═══════════════════╪═════════════════════╡
│ 2004 ┆ 4       ┆ 1       ┆ 38.398011         ┆ 25                  │
│ 2005 ┆ 4       ┆ 1       ┆ 38.049643         ┆ 25                  │
│ 2006 ┆ 2       ┆ 1       ┆ 37.584            ┆ 55                  │
│ 2010 ┆ 4       ┆ 1       ┆ 38.229102         ┆ 25                  │
│ 2011 ┆ 2       ┆ 1       ┆ 37.648676         ┆ 25                  │
│ …    ┆ …       ┆ …       ┆ …                 ┆ …                   │
│ 2006 ┆ 1       ┆ 1       ┆ 38.198405         ┆ 55                  │
│ 2012 ┆ 2       ┆ 1       ┆ 37.883794         ┆ 25                  │
│ 2011 ┆ 4       ┆ 1       ┆ 38.575118         ┆ 25           

## Comparison of Sequential and Parallel

Overall, parallel composition provides a much better bound than sequential composition. The total privacy loss for parallel composition is just ε since there are $k$ disjoint partitions and each of them also satisfies ε-dp. With sequential composition, the total privacy loss is $k$ * ε since we would run each query $k$ times.

However, one disadvantage of parallel composition is that has the dataset is split into more parts, each part will have a weaker signal and hence less accuracy. 

Additionally, in cases where we want statistics that reference the whole dataset, sequential composition may be a better fit. 

Therefore, the recommendation is to use parallel composition unless the partitions are too small to provide adequate accuracy or the entire data needs to be referenced for the calculated statistics. 

## Conclusion

This notebook illustrated: 
- The difference between interactive and noninteractive queries. 
- The definition and properties of sequential composition. 
- How to implement noninteractive sequential composition with polars, interactive sequential composition using transformations and measurements directly, and interactive sequential composition using polars queries. 
- The definition and properties of parallel composition. 
- How to implement noninterative parallel composition and interactive parallel composition. 
- The differences between parallel and sequential composition and recommendations for when to choose which. 
